In [ ]:
import implicit
import pandas
from scipy.sparse import csr_matrix
import os
os.chdir("/Users/Konstantin/Downloads/yoochoose/")
from tqdm.auto import tqdm
tqdm.pandas("Progress")

## Load Data

Загрузим датасет с покупками от **Yoochoose**.

- 1.1M покупок
- 20K товаров

In [102]:
buys = pandas.read_csv("buys.dat", names=['uid','ts','itemid','price','quantity'])[['uid','ts','itemid']]
buys.head()

,uid,ts,itemid
0,420374,2014-04-06T18:44:58.314Z,214537888
1,420374,2014-04-06T18:44:58.325Z,214537850
2,281626,2014-04-06T09:40:13.032Z,214535653
3,420368,2014-04-04T06:13:28.848Z,214530572
4,420368,2014-04-04T06:13:28.858Z,214835025


Let's generate some personalization Data

In [103]:
import random
buys['region'] = buys.uid.apply(lambda x: random.choice(['MSK','SPB','QWE','WER','ERT']))
buys['gender'] = buys.uid.apply(lambda x: random.choice(['M','F']))
buys['dow'] = buys.uid.apply(lambda x: random.choice(['MON','TUE','WED','THU','FRI','SAT','SUN']))
buys['type'] = "checkout"

## Popular Model

In [ ]:
from collections import Counter

n_popular = 5

popular_items = buys.groupby(['type','region','gender','dow'], as_index=False).\
  apply(lambda x: [x[0] for x in Counter(x['itemid']).most_common(n_popular)]).\
  to_frame(name="popular_items").reset_index()

## Regular Model

In [ ]:
n_popular = 2
regular_items = buys.groupby("uid", as_index=False).\
  progress_apply(lambda x: [x[0] for x in Counter(x['itemid']).most_common(n_popular)]).to_frame(name="regular_items").reset_index()

Все эти дурацкие библиотеки работают только с матричным представлением рейтингов, поэтому нужно сконверитровать в формат user-item матрицы

In [104]:
ratings = buys[['uid','itemid']].drop_duplicates()
ratings['quantity'] = 1

С помощью pandas.Pivot не получится, так как он сразу возвращает dense-представление и оно всю память съедает. Нужно руками собирать CSR матрицу.

In [ ]:
from pandas.api.types import CategoricalDtype
userType = CategoricalDtype(ratings.sessionid.unique(), ordered=True)
itemType = CategoricalDtype(ratings.itemid.unique(), ordered=True)

In [ ]:
itemType.categories.size

In [ ]:
row = ratings.sessionid.astype(userType).cat.codes
col = ratings.itemid.astype(itemType).cat.codes
ratings = csr_matrix((ratings["quantity"], (row, col)), shape=(userType.categories.size, itemType.categories.size))

**Далее запускать на instam-mg2!**

## Association Rules

In [ ]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)

## Matrix Factorization

Попробуем разложить в произведение факторных описаний

In [ ]:
from implicit.als import AlternatingLeastSquares
model = AlternatingLeastSquares()
model.fit(ratings)

In [ ]:
# buys.head()
model.recommend(0, ratings, N=5, filter_already_liked_items=True)

## KNN

In [ ]:
from sklearn.neighbors import KDTree
ratings1 = random.random_sam
tree = KDTree(ratings, leaf_size=40)

In [ ]:
ratings.shape